## Creation de it_equipment -> it_equipment_v2

## Creation de asset_new -> asset_filtree_v2

In [ ]:
DROP TABLE IF EXISTS assets_filtree_v2;
CREATE TABLE assets_filtree_v2
as
select *
from`assets_new`
where (API = 'FRVLB0C2' or API = 'FRTCRAVA')

In [ ]:
ALTER TABLE assets_filtree_v2
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot varchar(255),
ADD COLUMN Slot_begin varchar(255),
ADD COLUMN Slot_end varchar(255);

In [ ]:
UPDATE assets_filtree_v2
SET 
	Rack = SUBSTRING_INDEX(DALLE, '/', 1)  ,
	Slot = SUBSTRING_INDEX(DALLE, '/', -1) ,
	Slot_begin = SUBSTRING_INDEX(SUBSTRING_INDEX(DALLE, '/', -1), '-', 1) ,
	Slot_end = SUBSTRING_INDEX(DALLE, '-', -1)

## Creation de olivier_it

In [ ]:
ALTER TABLE olivier_it
ADD COLUMN STATUS_2 varchar(255),
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot_u varchar(255),
ADD COLUMN STATUS_3 varchar(255),
ADD COLUMN ASSET_ID_OK_POSITION varchar(255) ;

In [ ]:
UPDATE `olivier_it`
set
Nom_de_baie = SUBSTRING_INDEX(reperage_id,':',1) ,
Slot_u = SUBSTRING_INDEX(reperage_id,'Slot',-1) 

# It_equipment

In [ ]:
ALTER VIEW IT_Equipment_match_position_reduced_v2
as
SELECT it.reperage_id, count(it.reperage_id) as RowNumber

FROM   `IT_Equipment_v2` it 
INNER JOIN `assets_filtree_v2`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
GROUP BY it.reperage_id

In [ ]:
ALTER VIEW IT_Equipment_match_position_v2
AS
SELECT it.asset_id asset_id_it,
       it.short_name,
       it.nom,
       it.status   status_it,
       it.modele,
       it.reperage_id,
       it.hauteur_u,
       aa.ASSET_ID ASSET_ID_aa, aa.NOM_COURT NOM_COURT_aa, aa.NOM_LONG NOM_LONG_aa, aa.FABRICANT FABRICANT_aa, aa.MODEL MODEL_aa, aa.DALLE DALLE_aa, aa.SALLE SALLE_aa, aa.etat, it2.RowNumber

       
FROM   `IT_Equipment_v2` it 
INNER JOIN `assets_filtree_v2`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
INNER JOIN `IT_Equipment_match_position_reduced_v2` it2
ON it.reperage_id = it2.reperage_id

WHERE it2.rowNumber > 1
ORDER BY it2.rowNumber desc, Reperage_ID, it.status ASC

In [ ]:
alter VIEW it_built_system_NAS_v2
AS
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment_v2` it
WHERE  status IS NOT NULL and (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%' ) and ASSET_ID != '0'
ORDER  BY nom_de_baie DESC

In [ ]:
alter VIEW it_build_system_EXADATA_v2
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `IT_Equipment_v2` it
where it.fabricant like '%oracle%' and asset_id != '0'
ORDER  BY nom_de_baie DESC

In [ ]:
alter VIEW it_build_system_HPC_v2
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID,
        (LEFT(Nom,6)) ROOT_FERME
FROM   `IT_Equipment_v2` it
where it.nom like '%HPC%' or it.long_name like '%HPC%'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

In [ ]:
alter VIEW it_build_system_HANNA_v2
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
        
FROM   `IT_Equipment_v2` it
where (it.nom_de_baie like '%hana%' OR it.nom like '%hana%')  and asset_id != '0'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

In [ ]:
alter VIEW it_build_system_RING_v2
as
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID
       
       
FROM   `IT_Equipment_v2` it
WHERE (it.short_name like '%NA9TCR%' 
OR it.nom like '%NA9TCR%') and (it.status is not null or it.asset_id != '0')	
GROUP BY nom_de_baie, nom_de_la_salle
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

In [ ]:
alter VIEW IT_Equipment_FEX_v2
as
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID,
        (LEFT(it.Nom,8)) ROOT_FEX
from `IT_Equipment_v2` it
where Fabricant	 like '%cisco%' and ( it.nom = (LEFT(it.Nom,8)) )
ORDER BY Reperage_ID asc

In [ ]:
alter VIEW IT_Equipment_OLD_v2
AS
SELECT it.nom_de_baie,
		aa.nom_court,
        it.short_name,
        aa.nom_long,
        it.nom,
        long_name,
       it.nom_de_la_salle,
       it.status,
       REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','') new_nom_court,
		REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','') new_nom_long
        

from `IT_Equipment_v2` it
LEFT JOIN `assets_filtree_v2` aa
ON REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name

where aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD'

In [ ]:
alter VIEW IT_Equipment_cleaning_Hitachi_v2
as
SELECT it.nom_de_baie,
        it.short_name it_short_name,
        it.nom it_nom,
       it.nom_de_la_salle,
       it.status,
       (CASE WHEN LOCATE(':',it.nom) > 0 then 
       UPPER(REPLACE(REPLACE(LEFT(it.nom, LOCATE(':',it.nom)-1),'_','-'),'-',''))
       else it.nom
       END) clean_name

from `IT_Equipment_v2` it 
WHERE Fabricant like '%Hitachi%'
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

In [ ]:
-- 1988 rows affected
UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_v2` aa
ON aa.asset_id = it.asset_id
and it.asset_id != '0'
SET it.status = 'OK-CMDB-ASSET-ID'

### `Asset_ID affiché`

In [ ]:
UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_v2` aa on LEFT(aa.nom_court, length(aa.nom_court) - 3) = it.short_name
where it.type = 'Serveurs' and it.status is null

### `OK-POSITION`

In [ ]:
-- 596
UPDATE `IT_Equipment_v2` it 
INNER JOIN `assets_filtree_v2`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v2` it2
ON it.reperage_id = it2.reperage_id

SET it.status = Concat('OK-POSITION:', it2.rowNumber)

WHERE it.status is null AND (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

## HERE

In [ ]:
-- 115 rows affected
UPDATE   `IT_Equipment_v2` it
 LEFT JOIN `it_built_system_NAS_v2` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status = ( 
    			CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  	WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END )
                
WHERE it.status is NULL 
AND (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%')

In [ ]:
-- 61 rows affected
UPDATE   `IT_Equipment_v2` it
 LEFT JOIN `it_build_system_EXADATA_v2` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status = ( 
    			CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  	WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END )
                
WHERE it.status is NULL 
AND (it.fabricant like '%oracle%')

In [ ]:
-- 45 rows affected
UPDATE `IT_Equipment_v2` it
 LEFT JOIN `it_build_system_HPC_v2` it2
 ON (LEFT(it.Nom,6)) = it2.root_ferme

SET it.status = ( 
    			CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.root_ferme) 
        	  	WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END )

WHERE (it.nom like '%HPC%' 
OR it.long_name like '%HPC%') and it.status is NULL

In [ ]:
-- 72 ROWS
UPDATE `IT_Equipment_v2` it
  LEFT JOIN `it_build_system_HANNA_v2` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status =(CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
				WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END ) 

WHERE it.status is null and (it.nom_de_baie like '%hana%' OR it.nom like '%hana%')

In [ ]:
-- 9 rows
UPDATE `IT_Equipment_v2` it
  LEFT JOIN `it_build_system_RING_v2` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status =(CASE WHEN it.status is null and it2.status is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
				WHEN it.status is null and it2.status is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END ) 

WHERE it.status is NULL and (it.short_name like '%NA9TCR%' 
OR it.nom like '%NA9TCR%')

In [ ]:
-- 61 rows
UPDATE `IT_Equipment_v2` it
SET it.status = 'PCP : No Match'
where (it.Nom like '%_Rack%' or it.short_name like '%_Rack%') and (asset_id is null or asset_id ='0') and status is null

In [ ]:
-- 11 rows
UPDATE `IT_Equipment_v2` it
LEFT JOIN `IT_Equipment_FEX_v2` it2 on it2.root_fex = (LEFT(it.Nom,8)) and it.Nom != it2.root_fex

SET it.status =(CASE WHEN it.status is null and it2.status is not null and it2.asset_id !='0' then Concat(it.fabricant, ' : ', it2.asset_id) 
                WHEN it.status is null and it2.status is null and (it.asset_id is null or it2.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
                END ) 

where it.Fabricant like '%cisco%' and it2.root_fex is not null and it.status is null

In [ ]:
-- 69 rows
UPDATE `IT_Equipment_v2` it
LEFT JOIN `assets_filtree_v2` aa
ON REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name

SET it.status = 'CMDB-OK-OLD'

where it.status is NULL and (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD')

In [ ]:
-- 116 rows
UPDATE `IT_Equipment_v2` it
INNER JOIN `IT_Equipment_cleaning_Hitachi_v2` hit on hit.it_nom = it.nom
LEFT JOIN `assets_filtree_v2` aa on UPPER(REPLACE(REPLACE(aa.nom_court,'_',''),'-','')) = hit.clean_name

SET it.status = (CASE WHEN aa.asset_id is null and it.status is null then 'Hitashi : No match'
    			WHEN aa.asset_id is not null and it.status is null then concat('HDS : ',aa.asset_id)
    			END 
    			)
WHERE it.Fabricant like '%Hitachi%'


In [ ]:
-- 0 rows
UPDATE `IT_Equipment_v2` it 

LEFT JOIN 
	`assets_filtree_v2` aa 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)

SET it.status = "MATCH-POSITION : Install ou Retired"

WHERE (modele like '%Proliant%' OR modele like '%C7000%' ) 
and (API = 'FRVLB0C2' or API = 'FRTCRAVA')
and (it.status is null OR it.status ='MISMATCH_TIRET')

In [ ]:
-- 225 rows
UPDATE `IT_Equipment_v2` it 

LEFT JOIN 
	`assets_filtree_v2` aa 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)

SET it.status = "SANS MATCH-POSITION : CONTROL SUR SITE"

WHERE (modele like '%Proliant%' OR modele like '%C7000%' ) 
and (it.status is null OR it.status ='MISMATCH_TIRET')
and aa.dalle is null -- dalle null = pas de match car i ls' agit d'un left join 

In [ ]:
-- 174 rows
UPDATE `IT_Equipment_v2` it 
SET it.status = "Omniswitch : Not in CMDB - To validate with Serge"
where modele like '%omniswitch%'

In [ ]:
-- 205 rows
UPDATE `IT_Equipment_v2` it 
SET it.status = "Patch Panel : No action expected"
where type like '%Patch Panel%'

---

# Olivier_it

## LEFT JOIN

In [ ]:
UPDATE `olivier_it` oit
INNER JOIN `IT_Equipment_v2` it on oit.olivier_index = CONCAT(it.Nom_de_la_salle, it.reperage_id)

SET oit.status_2 = 'Commun'

where it.nom_de_la_salle in ('P1', 'P2', 'P3', 'P4')

## RIGHT JOIN

In [ ]:
UPDATE `olivier_it` oit
LEFT JOIN `IT_Equipment_v2` it on oit.olivier_index = CONCAT(it.Nom_de_la_salle, it.reperage_id)

set oit.status = 'Non Trouvé'

where  it.reperage_id is null

In [ ]:
DROP TABLE IF EXISTS assets_filtree_reduced ;
CREATE TABLE assets_filtree_reduced
AS
SELECT ype, subtype, ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, No_serie, etat, api, tag_partenaire
from `assets_filtree_v2` aa
WHERE SALLE in ('P1', 'P2', 'P3', 'P4', '0')

In [ ]:
ALTER VIEW IT_Equipment_match_position_reduced_v3
as
SELECT it.reperage_id, count(it.reperage_id) as RowNumber,
		concat(aa.asset_id) asset_id

FROM   `olivier_it` it  
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
GROUP BY it.reperage_id

In [ ]:
ALTER VIEW IT_Equipment_match_position_v3
AS
SELECT it.asset_id asset_id_it,
       it.short_name,
       it.nom,
       it.status_2   status_it,
       it.modele,
       it.reperage_id,
       aa.ASSET_ID ASSET_ID_aa, aa.NOM_COURT NOM_COURT_aa, aa.NOM_LONG NOM_LONG_aa, aa.FABRICANT FABRICANT_aa, aa.MODEL MODEL_aa, aa.DALLE DALLE_aa, aa.SALLE SALLE_aa, it2.RowNumber

       
FROM   `olivier_it` it 
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
INNER JOIN `IT_Equipment_match_position_reduced_v3` it2
ON it.reperage_id = it2.reperage_id

WHERE it2.rowNumber > 1
ORDER BY it2.rowNumber desc, Reperage_ID, it.status_2 ASC

In [ ]:
ALTER VIEW it_built_system_NAS_v3
AS
SELECT nom_de_baie,
       nom_de_la_salle,
       status_2,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `olivier_it` it
WHERE  status_2 IS NOT NULL and (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%' ) and ASSET_ID != '0' and ASSET_ID != '' 
ORDER  BY nom_de_baie DESC

In [ ]:
ALTER VIEW it_build_system_EXADATA_v3
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status_2,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
FROM   `olivier_it` it
where it.fabricant like '%oracle%' and asset_id != '0'
ORDER  BY nom_de_baie DESC

In [ ]:
ALTER VIEW it_build_system_HPC_v3
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status_2,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID,
        (LEFT(Nom,6)) ROOT_FERME
FROM   `olivier_it` it
where it.nom like '%HPC%' or it.short_name like '%HPC%'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

In [ ]:
ALTER VIEW it_build_system_HANNA_v3
as
SELECT nom_de_baie,
       nom_de_la_salle,
       status_2,
       asset_id,
       nom, 
        Modele,
        fabricant,
        Reperage_ID
        
FROM   `olivier_it` it
where (it.nom_de_baie like '%hana%' OR it.nom like '%hana%')  and asset_id != '0'
ORDER BY Nom_de_baie DESC, Nom_de_la_salle

In [ ]:
ALTER VIEW it_build_system_RING_v3
as
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status_2,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID
       
       
FROM   `olivier_it` it
WHERE (it.short_name like '%NA9TCR%' 
OR it.nom like '%NA9TCR%') and (it.status_2 is not null or it.asset_id != '0')	
GROUP BY nom_de_baie, nom_de_la_salle
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

In [ ]:
ALTER VIEW IT_Equipment_FEX_v3
as
SELECT it.nom_de_baie,
       it.nom_de_la_salle,
       it.status_2,
       it.asset_id,
       it.nom, 
        it.Modele,
        it.fabricant,
        it.Reperage_ID,
        (LEFT(it.Nom,8)) ROOT_FEX
from `olivier_it` it
where Fabricant	 like '%cisco%' and ( it.nom = (LEFT(it.Nom,8)) )
ORDER BY Reperage_ID asc

In [ ]:
ALTER VIEW IT_Equipment_OLD_v3
AS
SELECT it.nom_de_baie,
		aa.nom_court,
        it.short_name,
        aa.nom_long,
        it.nom,
       it.nom_de_la_salle,
       it.status_2,
       REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','') new_nom_court,
		REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','') new_nom_long
        

from `olivier_it` it
LEFT JOIN `assets_filtree_reduced` aa
ON REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name

where aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD'

In [ ]:
ALTER VIEW IT_Equipment_cleaning_Hitachi_v3
as
SELECT it.nom_de_baie,
        it.short_name it_short_name,
        it.nom it_nom,
       it.nom_de_la_salle,
       it.status_2,
       (CASE WHEN LOCATE(':',it.nom) > 0 then 
       UPPER(REPLACE(REPLACE(LEFT(it.nom, LOCATE(':',it.nom)-1),'_','-'),'-',''))
       else it.nom
       END) clean_name

from `olivier_it` it 
WHERE Fabricant like '%Hitachi%'
ORDER BY it.Nom_de_baie DESC, it.Nom_de_la_salle

In [ ]:
 ALTER VIEW it_build_system_DXI 
as
SELECT it.asset_id asset_id_it, it.short_name, it.nom, it.status_2 status_it, it.modele, it.reperage_id , Nom_de_la_salle, Nom_de_baie, left(it.nom,9) DXI_ROOT
FROM `olivier_it` it 
where modele like '%dxi%' 
 and ( it.nom = left(it.nom,9) or it.short_name = left(it.nom,9) )

In [ ]:
-- 1988 rows affected
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON aa.asset_id = it.asset_id
and it.asset_id != '0'
SET it.status_2 = 'OK-CMDB-ASSET-ID'

In [ ]:
-- 250
UPDATE `olivier_it` it 
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3` it2
ON it.reperage_id = it2.reperage_id

SET it.status_2 = Concat('OK-POSITION:', it2.rowNumber)

WHERE it.status_2 is null AND (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

In [ ]:
-- 42 rows affected
UPDATE   `olivier_it` it
 LEFT JOIN `it_built_system_NAS_v3` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status_2 = ( 
    			CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  	WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0' or it.asset_id ='') then  Concat(it.fabricant, ' : No Match') 
        		END )
                
WHERE it.status_2 is null
AND (it.fabricant like 'netapp%' or it.fabricant like 'ATTO%' or it.fabricant like 'Brocade%')

In [ ]:
-- 3 rows affected
UPDATE   `olivier_it` it
 LEFT JOIN `it_build_system_EXADATA_v3` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status_2 = ( 
    			CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
        	  	WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END )
                
WHERE it.status_2 is NULL 
AND (it.fabricant like '%oracle%')

In [ ]:
-- 0 rows affected
UPDATE `olivier_it` it
 LEFT JOIN `it_build_system_HPC_v3` it2
 ON (LEFT(it.Nom,6)) = it2.root_ferme

SET it.status_2 = ( 
    			CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.root_ferme) 
        	  	WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END )

WHERE (it.nom like '%HPC%' 
OR it.short_name like '%HPC%') and it.status_2 is NULL

In [ ]:
-- 0 ROWS
UPDATE `olivier_it` it
  LEFT JOIN `it_build_system_HANNA_v3` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status_2 =(CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
				WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END ) 

WHERE it.status_2 is null and (it.nom_de_baie like '%hana%' OR it.nom like '%hana%')

In [ ]:
-- 0 rows
UPDATE `olivier_it` it
  LEFT JOIN `it_build_system_RING_v3` it2
 ON        it2.nom_de_baie = it.nom_de_baie 
 AND       it2.nom_de_la_salle = it.nom_de_la_salle
 AND		  it2.fabricant = it.fabricant

SET it.status_2 =(CASE WHEN it.status_2 is null and it2.status_2 is not null then Concat(it.fabricant, ' : ', it2.asset_id) 
				WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
        		END ) 

WHERE it.status_2 is NULL and (it.short_name like '%NA9TCR%' 
OR it.nom like '%NA9TCR%')

In [ ]:
-- 0 rows
UPDATE `olivier_it` it
SET it.status_2 = 'PCP : No Match'
where (it.Nom like '%_Rack%' or it.short_name like '%_Rack%') and (asset_id is null or asset_id ='0') and status_2 is null

In [ ]:
-- 13 rows
UPDATE `olivier_it` it
LEFT JOIN `IT_Equipment_FEX_v3` it2 on it2.root_fex = (LEFT(it.Nom,8)) and it.Nom != it2.root_fex

SET it.status_2 =(CASE WHEN it.status_2 is null and it2.status_2 is not null and it2.asset_id !='0' then Concat(it.fabricant, ' : ', it2.asset_id) 
                WHEN it.status_2 is null and it2.status_2 is null and (it.asset_id is null or it2.asset_id ='0') then  Concat(it.fabricant, ' : No Match') 
                END ) 

where it.Fabricant like '%cisco%' and it2.root_fex is not null and it.status_2 is null

In [ ]:
-- 903 rows !?????
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name
AND (it.SHORT_NAME != '' )

SET it.status_2 = 'CMDB-OK-OLD'


where it.status_2 is null and (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD')


In [ ]:
-- 87 rows
UPDATE `olivier_it` it
INNER JOIN `IT_Equipment_cleaning_Hitachi_v3` hit on hit.it_nom = it.nom
LEFT JOIN `assets_filtree_reduced` aa on UPPER(REPLACE(REPLACE(aa.nom_court,'_',''),'-','')) = hit.clean_name

SET it.status_2 = (CASE WHEN aa.asset_id is null and it.status_2 is null then 'Hitashi : No match'
    			WHEN aa.asset_id is not null and it.status_2 is null then concat('HDS : ',aa.asset_id)
    			END 
    			)
WHERE it.Fabricant like '%Hitachi%'


In [ ]:
-- 0 rows
UPDATE `olivier_it` it 

LEFT JOIN 
	`assets_filtree_reduced` aa 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)

SET it.status_2 = "MATCH-POSITION : Install ou Retired"

WHERE (modele like '%Proliant%' OR modele like '%C7000%' ) 
and (API = 'FRVLB0C2' or API = 'FRTCRAVA')
and (it.status_2 is null OR it.status_2 ='MISMATCH_TIRET')

In [ ]:
-- 189 rows
UPDATE `olivier_it` it 

LEFT JOIN 
	`assets_filtree_reduced` aa 
ON 
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)

SET it.status_2 = "SANS MATCH-POSITION : CONTROL SUR SITE"

WHERE (modele like '%Proliant%' OR modele like '%C7000%' ) 
and (it.status_2 is null OR it.status_2 ='MISMATCH_TIRET')
and aa.dalle is null -- dalle null = pas de match car i ls' agit d'un left join 

In [ ]:
-- 174 rows
UPDATE `olivier_it` it 
SET it.status_2 = "Omniswitch : Not in CMDB - To validate with Serge"
where modele like '%omniswitch%'

In [ ]:
-- 106 rows
UPDATE `olivier_it` it 
SET it.status_2 = "Patch Panel : No action expected"
where type like '%Patch Panel%'

In [ ]:
-- 15 rows
UPDATE `olivier_it` it 
LEFT JOIN `it_build_system_DXI` it2 ON left(it.nom,9) = it2.DXI_ROOT
SET it.status_2 = (CASE WHEN it.status_2 is null and it2.status_it is null then 'DXI : No match'
					WHEN it.status_2 is null and it2.status_it is not null then concat('DXI : ',it2.asset_id_it)
					END 
					)
                    
where it.modele like '%dxi%' and status_2 is null

# `NEW STATUS_3`

In [ ]:
-- v2 : 2070
UPDATE olivier_it

SET status_3 = 	CASE WHEN status_2 = 'OK-CMDB-ASSET-ID' then 'OK-CMDB-ASSET-ID'
		WHEN status_2 != 'OK-CMDB-ASSET-ID' and status_2 is not null then 'Identifiable'
        WHEN status_2 is null then NULL
        END

# `OK-POSITION : 1` 

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3` it2
ON it.reperage_id = it2.reperage_id

SET it.ASSET_ID_OK_POSITION = it2.asset_id

WHERE it.status_2 = 'OK-POSITION:1'